## Programmatic Access: Submit and Retrieve Artifacts using the SDMX Lab

This notebook demonstrates how to submit and retrieve artifacts using REST API calls to FMR running inside an SDMX Lab instance.

In [ ]:
from pathlib import Path
import requests
import certifi
from lxml import etree
from getpass import getpass
from requests.auth import HTTPBasicAuth

Set up your credentials for basic authorization. These will be used to authenticate REST API calls to the Lab instance.

In [ ]:
user = input("Username: ")
password = getpass("Password: ")
auth = HTTPBasicAuth(username=user, password=password)

Specify the URL endpoint for the Lab instance. This is where the REST API calls will be sent.

In [ ]:
# Define Lab instance endpoint
# This is constructed automatically from the provided user name.
fmr_url = f"https://{user}.sdmx.solutions/fmr"
endpoint = f"{fmr_url}/ws/secure/sdmxapi/rest"

### Submit a dataflow to FMR via REST-API

This step demonstrates how to submit a dataflow to FMR using the REST endpoint.

In [ ]:
# Prepare SDMX resource
path = 'example_dataflow.xml'
dataflow = Path(path).read_text()
headers = {"Content-Type": "application/xml"}

response = requests.post(
    endpoint, 
    data=dataflow, 
    headers=headers, 
    auth=(user, password),
    verify=False
)

In [ ]:
# Display response
if response.status_code in [200, 201]:
    print("Resource successfully created in FMR.")
    print("Response:", response.text)
else:
    print(f"Failed to create resource. Status code: {response.status_code}")
    print("Response:", response.text)

### Retrieve DSDs in submitted dataflow from FMR via REST-API

This step demonstrates how to retrieve the individual DSDs in the submitted dataflow from FMR using the REST endpoint.

In [ ]:
endpoint_base = fmr_url + "/sdmx/v2/structure/datastructure"
root = etree.fromstring(dataflow.encode('utf-8'))

resource_urls = []
for dsd in root.findall(".//str:DataStructure", namespaces=root.nsmap):
    attributes = {key.lower(): value for key, value in dsd.attrib.items()}
    endpoint = f"{endpoint_base}/{attributes['agencyid'].strip()}/{attributes['id'].strip()}/{attributes['version'].strip()}"
    response = requests.get(endpoint, headers=headers, auth=auth, verify=False)
    print(f"Resource successfully retrieved. Status code: {response.status_code}.")
    print("Response:", response.text)